In [1]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions
#from bs4 import BeautifulSoup
#import requests
#import urllib.request,sys,time
from time import sleep

In [2]:
# read the website
driver = webdriver.Chrome()
website = "https://www.weforum.org/agenda/climate-change"
driver.get(website)
sleep(5)

In [3]:
# scrape final page number
pagenum = driver.find_element(By.XPATH, '//p[@class="chakra-text wef-1u0z722"]')
finalpagenum = pagenum.text.replace('1 / ', '')

In [4]:
finalpagenum

'337'

In [5]:
# scrape article links from multipaginated website

columns = ['Article Name', 'Link', 'Author(s)', 'Date', 'Category']
websitedf = pd.DataFrame(columns=columns)

for num in range(1, int(finalpagenum)+1):
    url = website + '?page=' + str(num)
    
    # read the website
    driver.get(url)
    sleep(5)
    
    # scrape article names
    # for first two articles
    articles1 = driver.find_elements(By.XPATH, '//p[@class="chakra-text wef-1nxztse"]')
    articles1_list = []
    for article in range(len(articles1)):
        articles1_list.append(articles1[article].text)

    # for rest of the articles
    articles2 = driver.find_elements(By.XPATH, '//p[@class="chakra-text wef-78x0fz"]')
    articles2_list = []
    for article in range(len(articles2)):
        articles2_list.append(articles2[article].text)

    # join the two articles list
    article_list = articles1_list + articles2_list
    
    # scrape article links
    # for first two articles
    links1 = driver.find_elements(By.XPATH, '//p[@class="chakra-text wef-1nxztse"]/a')
    links1_list = []
    for link in links1:
        links1_list.append(link.get_attribute('href'))

    # for rest of articles
    links2 = driver.find_elements(By.XPATH, '//p[@class="chakra-text wef-78x0fz"]/a')
    links2_list = []
    for link in links2:
        links2_list.append(link.get_attribute('href'))

    links_list = links1_list + links2_list
    
    # scrape author names
    authors = driver.find_elements(By.XPATH, '//p[@class="chakra-text wef-kefhaq"]')
    authors_list = []
    for author in range(len(authors)):
        authors_list.append(authors[author].text)
    
    # scrape dates
    dates = driver.find_elements(By.XPATH, '//p[@class="chakra-text wef-1iho44l"]')

    # list of dates
    dates_list = []
    for date in range(len(dates)):
        dates_list.append(dates[date].text)
        
    # scrape the archive category
    category = driver.find_elements(By.XPATH, '//a[@class="chakra-badge wef-amhzg0"]')
    category_list = []
    for cat in range(len(category)):
        category_list.append(category[cat].text)
        
    # store data in to a dataframe
    data = list(zip(article_list, links_list, authors_list, dates_list, category_list))
    df = pd.DataFrame(data, columns=['Article Name', 'Link', 'Author(s)', 'Date', 'Category'])
    df['Date'] = df['Date'].apply(pd.to_datetime)
    
    # add the df into the websitedf
    websitedf = websitedf.append(df, ignore_index=True)

In [6]:
websitedf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5052 entries, 0 to 5051
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Article Name  5052 non-null   object        
 1   Link          5052 non-null   object        
 2   Author(s)     5052 non-null   object        
 3   Date          5052 non-null   datetime64[ns]
 4   Category      5052 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 197.5+ KB


In [7]:
# loop through all the links and scrape article content
# then save the content into csv

contents = []

for link in websitedf['Link']:
    driver.get(link)
    sleep(5)
    
    # scrape article content
    paragraph = driver.find_elements(By.XPATH, '//div[@class="wef-foxfue"]')
    allparagraph = []
    for para in range(len(paragraph)):
        allparagraph.append(paragraph[para].text)
    
    contents.append(allparagraph)

websitedf['Contents'] = contents

In [8]:
websitedf.to_excel('WEF Climate Change.xlsx')

In [2]:
# load the file
climatechange = pd.read_excel('WEF Climate Change.xlsx', index_col=0)

In [3]:
missingcontentdf = climatechange[climatechange['Contents'] == '[]']

driver = webdriver.Chrome()
missingcontent = []
for link in missingcontentdf['Link']:
    # scrape article content
    driver.get(link)
    sleep(1)
    paragraph = driver.find_elements(By.XPATH, '//div[@class="wef-11zt3rm"] | //div[@class="st__content-block st__content-block--text"] | //div[@class="wef-6mvns2"]')
    allparagraph = []
    for para in range(len(paragraph)):
        allparagraph.append(paragraph[para].text)
    
    missingcontent.append(allparagraph)

missingcontentdf['Contents'] = missingcontent

C:\Users\Wei'En\AppData\Local\Temp/ipykernel_25832/2779335578.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missingcontentdf['Contents'] = missingcontent


In [6]:
climatechange.update(missingcontentdf)

In [7]:
climatechange.to_excel('WEF Climate Change (updated).xlsx')